参考文章
https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/?utm_medium=social&utm_source=twitter.com&utm_campaign=buffer
实现了几种推荐算法，包括：itemCF, userCF, LFM


In [17]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import pairwise_distances

## 获取数据

In [2]:
def get_ratings(path, sep='\t', columns=None):
    return pd.read_table(path, sep=sep, names=columns)

In [14]:
ratings = get_ratings('./input/ml-100k/u.data', columns=['user_id', 'item_id', 'rating', 'timestamp'])
n_users = ratings['user_id'].unique().shape[0]
n_items = ratings['item_id'].unique().shape[0]
data_matrix = np.zeros((n_users, n_items))

## 相似度计算 —— userCF, itemCF

In [16]:
for idx, user_id, item_id, rating, ts in ratings.itertuples():
    data_matrix[user_id-1, item_id-1] = rating

In [33]:
# 计算用户向量和物品向量间的余弦相似度
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

## 预测

In [31]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [34]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [38]:
mean_user_rating = data_matrix.mean(axis=1)

In [41]:
mean_user_rating.shape

(943,)

In [42]:
data_matrix - mean_user_rating

ValueError: operands could not be broadcast together with shapes (943,1682) (943,) 

In [40]:
(data_matrix - mean_user_rating[:, np.newaxis]).shape

(943, 1682)